***
# Haversine and Geographic Distance
***

Locations are often specified as latitude/longitude pairs.

![coordinate_precision](https://imgs.xkcd.com/comics/coordinate_precision.png)

How can we calculate the distance between two geographic coordinates? The distance between lines of longitude (meridians) varies with latitude.

![latitude and longitude](https://upload.wikimedia.org/wikipedia/commons/e/ef/FedStats_Lat_long.svg)

A simple approximation is to calculate two-dimensional distance based on arc length ($S = r \theta$) for latitude (ϕ) and longitude (λ) in radians.
$$
\begin{aligned}
r &= 6371 && \text{average earth radius in km} \\
\Delta \phi &= \phi_2 - \phi_1 && \text{change in latitude} \\
\Delta \lambda &= \lambda_2 - \lambda_1 && \text{change in longitude} \\
\overline{\phi} &= \frac{\phi_1 + \phi_2}{2} && \text{average latitude} \\
d_x &= \Delta \lambda \cos{\overline{\phi}} && \text{E-W distance} \\
d_y &= \Delta \phi && \text{N-S distance} \\
d &= r \sqrt{d_x^2 + d_y^2}
\end{aligned}
$$

![great-circle distance](https://timeandnavigation.si.edu/sites/default/files/multimedia-assets/300-si_fl_great_circle_fa.jpg)

A better approximation is the great-circle distance on a sphere. The Earth is not a perfect sphere, but this approximation is accurate to within 1% and still fast to compute.
$$
\scriptsize
\begin{aligned}
d &= r\arccos{(\sin\phi_1\sin\phi_2 + \cos\phi_1\cos\phi_2\cos{\Delta \lambda})} % && \text{spherical law of cosines}
\\
  &= 2r\arcsin \sqrt{\sin^2\left(\frac{\Delta\phi}{2}\right) + \left(1 - \sin^2\left(\frac{\Delta\phi}{2}\right) - \sin^2\left(\frac{\phi_1 + \phi_2}{2}\right)\right)\cdot\sin^2\left(\frac{\Delta\lambda}{2}\right)} % && \text{Haversine formula}
\end{aligned}
$$

GIS software (e.g., QGIS, PostGIS, GeoPandas) implement more accurate but slower solutions referencing the **WGS84** ellipsoid model of the Earth. [Vincenty's iterative inverse method](https://en.wikipedia.org/wiki/Vincenty%27s_formulae) is accurate to within 1 mm, ignoring elevation differences.

Typically, GIS users instead **reproject** coordinates to an appropriate Cartesian plane for their geographic region, such as **[UTM zone 18N](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system)** or **[New York State Plane Long Island](https://en.wikipedia.org/wiki/State_Plane_Coordinate_System)**. Spatial indexes (bounding boxes) can also speed up subsequent lookups by limiting to computation to nearby coordinates.

## Computation

To evaluate several approaches for calculating distance, let's specify a reference location (`mylat`, `mylon`) corresponding to 433 Broadway. Then we'll create a pandas `DataFrame` containing 100,000 pairs of randomly-generated latitude/longitude coordinates to calculate the distance to.

In [1]:
import math
import numpy as np
import pandas as pd

R = 6371  # average earth radius in km
mylat = 40.72018
mylon = -74.00149
np.random.seed(42)
nrows = int(100e3)
df = pd.DataFrame(
    {
        "lat": np.random.uniform(40, 41, nrows),
        "lon": np.random.uniform(-74, -73, nrows),
    }
)
df

,lat,lon
0,40.374540,-73.419221
1,40.950714,-73.473028
2,40.731994,-73.648963
3,40.598658,-73.506787
4,40.156019,-73.634903
...,...,...
99995,40.792305,-73.377850
99996,40.779253,-73.370779
99997,40.674453,-73.807796
99998,40.499447,-73.346917


### Iterative Approach

We can iterate over each row to calculate Cartesian distance.

In [2]:
%%time
distances = []
for index, row in df.iterrows():
    dx = (math.radians(row["lon"]) - math.radians(mylon)) * math.cos(
        math.radians((row["lat"] + mylat) / 2)
    )
    dy = math.radians(row["lat"]) - math.radians(mylat)
    distances.append(R * math.sqrt(dx**2 + dy**2))
df["dist_cartesian_slow"] = distances

CPU times: user 3.58 s, sys: 2.78 ms, total: 3.58 s
Wall time: 3.59 s


Iteration is slow. Avoid iteration with large data if possible.

### Functional Approach

A better approach is to write a function and apply it row-wise with pandas. This is modular, easier to check for errors, and generally faster.

In [3]:
def cartesian(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dx = R * (lon2 - lon1) * math.cos((lat1 + lat2) / 2)
    dy = R * (lat2 - lat1)
    return math.sqrt(dx**2 + dy**2)

In [4]:
%%time
df["dist_cartesian"] = df[["lat", "lon"]].apply(
    lambda x: cartesian(x[1], x[0], mylon, mylat), axis=1
)

CPU times: user 634 ms, sys: 19.8 ms, total: 654 ms
Wall time: 654 ms


We can also use the haversine formula instead of a Cartesian approximation.

In [5]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    return 2 * R * math.asin(math.sqrt(a))

In [6]:
%%time
df["dist_haversine"] = df[["lat", "lon"]].apply(
    lambda x: haversine(x[1], x[0], mylon, mylat), axis=1
)

CPU times: user 684 ms, sys: 95 µs, total: 684 ms
Wall time: 683 ms


Even though there is more math, the computation time doesn't change that much. Why?

### Vectorization

The NumPy/pandas approach is to **vectorize**, replacing scalar values with **arrays** and native Python math functions with their vector equivalents. This is much faster for larger data sets and generally preferred.

In [7]:
def haversine_vec(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    a = (
        np.sin((lat2 - lat1) / 2) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2) ** 2
    )
    return 2 * R * np.arcsin(np.sqrt(a))

In [8]:
%%time
df["dist_haversine_vec"] = haversine_vec(df["lon"], df["lat"], mylon, mylat)

CPU times: user 10.1 ms, sys: 3 µs, total: 10.2 ms
Wall time: 11.9 ms


### Compilation

Another approach is to translate Python functions to optimized machine code with a just-in-time (JIT) compiler. This works with a subset of Python and NumPy code, including most loops and vectorized operations. The improvement is more drastic for loops, as vectorized operations are already pretty fast.

In [9]:
from numba import jit


@jit
def haversine_jit(lon1, lat1, lon2, lat2):
    # numba expects items passed to map to be the same data type.
    # lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)
    a = (
        np.sin((lat2 - lat1) / 2) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2) ** 2
    )
    return 2 * R * np.arcsin(np.sqrt(a))

In [10]:
%%timeit
df["dist_haversine_jit"] = haversine_jit(
    df["lon"].to_numpy(), df["lat"].to_numpy(), mylon, mylat
)

2.88 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Don't Reinvent the Wheel

The scientific Python ecosystem, including **NumPy**, **pandas**, and **SciPy**, often have many useful functions already written for us.

**scikit-learn**, a popular machine-learning library, implements a haversine function that we can use instead of our own.

In [11]:
from sklearn.metrics.pairwise import haversine_distances

In [12]:
%%time
df["dist_haversine_sk"] = R * haversine_distances(
    np.radians(df[["lat", "lon"]]), np.radians([[mylat, mylon]])
)

CPU times: user 14.7 ms, sys: 123 µs, total: 14.9 ms
Wall time: 13.5 ms


In [13]:
df

,lat,lon,dist_cartesian_slow,dist_cartesian,dist_haversine,dist_haversine_vec,dist_haversine_jit,dist_haversine_sk
0,40.374540,-73.419221,62.430507,62.430507,62.430249,62.430249,62.430249,62.430249
1,40.950714,-73.473028,51.319731,51.319731,51.319588,51.319588,51.319588,51.319588
2,40.731994,-73.648963,29.735655,29.735655,29.735635,29.735635,29.735635,29.735635
3,40.598658,-73.506787,43.862439,43.862439,43.862362,43.862362,43.862362,43.862362
4,40.156019,-73.634903,69.984428,69.984428,69.984241,69.984241,69.984241,69.984241
...,...,...,...,...,...,...,...,...
99995,40.792305,-73.377850,53.137565,53.137565,53.137445,53.137445,53.137445,53.137445
99996,40.779253,-73.370779,53.534139,53.534139,53.534018,53.534018,53.534018,53.534018
99997,40.674453,-73.807796,17.102472,17.102472,17.102468,17.102468,17.102468,17.102468
99998,40.499447,-73.346917,60.461590,60.461590,60.461374,60.461374,60.461374,60.461374


In [14]:
# This will raise an exception (error) if the values are different.
assert np.isclose(df["dist_cartesian_slow"], df["dist_cartesian"]).all()
assert np.isclose(df["dist_haversine"], df["dist_haversine_vec"]).all()
assert np.isclose(df["dist_haversine_vec"], df["dist_haversine_jit"]).all()
assert np.isclose(df["dist_haversine"], df["dist_haversine_sk"]).all()

## Practical Example with Open Data

Many government agencies provide data sets to the public, often as part of a request for public records (FOIL/FOIA) in "machine-readable format", through open data portals like [data.gov](https://data.gov/), or occasionally through **APIs**.

Some of this data is even personally identifiable, such as tax parcels and voter records. Journalists, political campaigns, mapping providers, weather forecasters, and the real estate industry all build on top of government data. You can too!

New York State and New York City both offer open data portals that contain data sets from their respective agencies:

 * [data.ny.gov](https://data.ny.gov/browse)
 * [data.cityofnewyork.us](https://data.cityofnewyork.us/browse)

To access NYS/NYC data we can go through the web portals and export/download the data we need.

Since the portals are powered by Socrata, we can also use the Socrata search engine [opendatanetwork.com](https://www.opendatanetwork.com/) to search for data sets or the Socrata Open Data API (SODA) to programmatically download data for us. `sodapy` is a Python library that implements a client for SODA.

### 311 Noise Complaints

![loud](https://imgs.xkcd.com/comics/loud_sex.png)

New York City is a loud city. It also has laws. If you have a noisy neighbor, you can call 311 and the cops will _in theory_ show up. Let's find the closest noise complaints to us.

NYC Open Data has a dataset called [311 Noise Complaints](https://data.cityofnewyork.us/Social-Services/311-Noise-Complaints/p5f6-bkga). On that page, click API. You should see the following information which we will use below:

> API Endpoint
>
>     https://data.cityofnewyork.us/resource/p5f6-bkga.json

In [15]:
from sodapy import Socrata

# Enter the information from the API endpoint above.
client = Socrata("data.cityofnewyork.us", app_token=None)
results = client.get("p5f6-bkga", where="created_date >= '2022-05-01'", limit=10000)
df = pd.DataFrame(results)

We specified the date using the format `YYYY-MM-DD`, also known as **ISO 8601** format.

This format is unambiguous for our international friends. More importantly for us, dates sort chronologically.

![iso_8601](https://imgs.xkcd.com/comics/iso_8601.png)

In [16]:
df.head()

,descriptor,incident_zip,created_date,location,city,cross_street_2,closed_date,park_facility_name,intersection_street_1,landmark,...,street_name,complaint_type,longitude,status,unique_key,y_coordinate_state_plane,resolution_action_updated_date,address_type,intersection_street_2,facility_type
0,Loud Music/Party,10452,2022-05-01T00:00:03.000,"{'latitude': '40.83065331987539', 'human_addre...",BRONX,EAST 165 STREET,2022-05-01T06:27:30.000,Unspecified,EAST 164 STREET,GERARD AVENUE,...,GERARD AVENUE,Noise - Residential,-73.92308231033165,Closed,54049402,241916,2022-05-01T06:27:35.000,ADDRESS,EAST 165 STREET,NaN
1,Loud Music/Party,10027,2022-05-01T00:00:09.000,"{'latitude': '40.81237834836466', 'human_addre...",NEW YORK,CONVENT AVENUE,2022-05-01T00:07:46.000,Unspecified,ST NICHOLAS TERRACE,WEST 127 STREET,...,WEST 127 STREET,Noise - Street/Sidewalk,-73.95240115039357,Closed,54046813,235252,2022-05-01T00:07:53.000,ADDRESS,CONVENT AVENUE,NaN
2,Loud Music/Party,10033,2022-05-01T00:00:17.000,"{'latitude': '40.854713655427595', 'human_addr...",NEW YORK,WEST 187 STREET,2022-05-01T00:10:38.000,Unspecified,WEST 185 STREET,FT WASHINGTON AVENUE,...,FT WASHINGTON AVENUE,Noise - Residential,-73.93702585036192,Closed,54045643,250679,2022-05-01T00:10:43.000,ADDRESS,WEST 187 STREET,NaN
3,Loud Music/Party,10031,2022-05-01T00:00:27.000,"{'latitude': '40.82824553459575', 'human_addre...",NEW YORK,WEST 155 STREET,2022-05-01T00:48:50.000,Unspecified,WEST 150 STREET,EDGECOMBE AVENUE,...,EDGECOMBE AVENUE,Noise - Residential,-73.94051254172763,Closed,54046765,241035,2022-05-01T00:48:54.000,ADDRESS,WEST 155 STREET,NaN
4,Loud Music/Party,10453,2022-05-01T00:00:32.000,"{'latitude': '40.848717168218855', 'human_addr...",BRONX,WEST 177 STREET,2022-05-01T10:56:54.000,Unspecified,WEST 176 STREET,DAVIDSON AVENUE,...,DAVIDSON AVENUE,Noise - Street/Sidewalk,-73.91278896581804,Closed,54048291,248500,2022-05-01T10:57:01.000,ADDRESS,WEST 177 STREET,NaN


There are 32 columns in the dataset. Let's pick only a few.

In [17]:
columns = [
    "descriptor",
    "created_date",
    "closed_date",
    "landmark",
    "resolution_description",
    "incident_address",
    "location_type",
    "agency",
    "complaint_type",
    "status",
    "unique_key",
    "resolution_action_updated_date",
    "address_type",
    "facility_type",
]

In [18]:
df[columns].head(3).T

,0,1,2
descriptor,Loud Music/Party,Loud Music/Party,Loud Music/Party
created_date,2022-05-01T00:00:03.000,2022-05-01T00:00:09.000,2022-05-01T00:00:17.000
closed_date,2022-05-01T06:27:30.000,2022-05-01T00:07:46.000,2022-05-01T00:10:38.000
landmark,GERARD AVENUE,WEST 127 STREET,FT WASHINGTON AVENUE
resolution_description,The Police Department responded to the complai...,The Police Department responded to the complai...,The Police Department responded to the complai...
incident_address,1006 GERARD AVENUE,364 WEST 127 STREET,593 FT WASHINGTON AVENUE
location_type,Residential Building/House,Street/Sidewalk,Residential Building/House
agency,NYPD,NYPD,NYPD
complaint_type,Noise - Residential,Noise - Street/Sidewalk,Noise - Residential
status,Closed,Closed,Closed


Let's calculate the distance to each complaint and sort by distance.

In [19]:
mylat, mylon

(40.72018, -74.00149)

In [20]:
df["distance"] = haversine_vec(
    df["longitude"].astype(float), df["latitude"].astype(float), mylon, mylat
)
df.sort_values("distance", inplace=True)

In [21]:
df[columns + ["distance"]].head(3).T

,7541,8618,9489
descriptor,Car/Truck Music,Noise: Construction Equipment (NC1),Loud Music/Party
created_date,2022-05-04T20:56:39.000,2022-05-05T11:20:00.000,2022-05-05T20:55:43.000
closed_date,2022-05-04T21:47:10.000,NaN,2022-05-05T21:55:15.000
landmark,MERCER STREET,NaN,MERCER STREET
resolution_description,The Police Department responded and upon arriv...,NaN,The Police Department responded to the complai...
incident_address,17 MERCER STREET,456 BROADWAY,26 MERCER STREET
location_type,Street/Sidewalk,NaN,Store/Commercial
agency,NYPD,DEP,NYPD
complaint_type,Noise - Vehicle,Noise,Noise - Commercial
status,Closed,Open,Closed


In [22]:
df["resolution_description"].iloc[0]

'The Police Department responded and upon arrival those responsible for the condition were gone.'

## Geocoding
 
Geocoding converts a text-based description of a location, such as the address `433 Broadway` or the name `Empire State Building` to geographic coordinates such as a latitude/longitude pair. Google Maps, Uber, Seamless, and many other services that accept an address convert it to a geographic location using an API. We can too!

The New York City Department of City Planning provides [NYC GeoSearch](https://geosearch.planninglabs.nyc/) as a free API service for locating addresses in the City.

In [23]:
import requests


def geocode_address(text):
    r = requests.get(
        "https://geosearch.planninglabs.nyc/v1/autocomplete",
        {"text": text, "size": 1},
    )
    point = r.json()["features"][0]
    print(point["properties"]["label"])
    return point["geometry"]["coordinates"][::-1]


geocode_address("433 Broadway Manhattan")

433 BROADWAY, Manhattan, New York, NY, USA


[40.720228, -74.001571]

In [24]:
geocode_address("Empire State Building")

THE EMPIRE STATE BUILDING, Manhattan, New York, NY, USA


[40.748433, -73.985654]

We can write another function that performs a geocoding lookup and uses the returned location to provide us the three closest complaints.

In [25]:
def get_closest_complaints(address):
    lat, lon = geocode_address(address)
    df2 = df.copy()
    df2["distance"] = haversine_vec(
        df2["longitude"].astype(float),
        df2["latitude"].astype(float),
        lon,
        lat,
    )
    df2.sort_values("distance", inplace=True)
    return df2[columns + ["distance"]].head(3).T

In [26]:
get_closest_complaints("staten island ferry")

STATEN ISLAND FERRY TERMINAL, Manhattan, New York, NY, USA


,6913,5963,3839
descriptor,Noise: Private Carting Noise (NQ1),Loud Music/Party,Loud Music/Party
created_date,2022-05-04T03:09:00.000,2022-05-03T16:18:33.000,2022-05-01T23:52:57.000
closed_date,NaN,2022-05-03T16:45:05.000,2022-05-02T06:44:39.000
landmark,NaN,BROAD STREET,WALL STREET
resolution_description,The Department of Environmental Protection has...,The Police Department responded to the complai...,The Police Department responded to the complai...
incident_address,10 HANOVER SQUARE,25 BROAD STREET,95 WALL STREET
location_type,NaN,Street/Sidewalk,Residential Building/House
agency,DEP,NYPD,NYPD
complaint_type,Noise,Noise - Street/Sidewalk,Noise - Residential
status,Started,Closed,Closed
